In [ ]:
from build_nba_bdd import export_all_seasons, export_all_teams_seasons, export_all_salaries, scrape_mvp_data, merge_data

In [ ]:
export_all_seasons()

In [ ]:
export_all_teams_seasons() # 5min environ

In [ ]:
export_all_salaries()

In [ ]:
scrape_mvp_data()

In [ ]:
merge_data() # Pour créer le data set sur lequel appliquer nos modèles (que pour Regular Season pour l'instant)

# Modèle de prédiction des salaires
Entraînement d'un modèle Random Forest pour prédire les salaires ajustés à l'inflation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Charger la base de données fusionnée
df = pd.read_csv('./data/merged_data.csv')

print(f"Dataset chargé: {len(df)} lignes, {len(df.columns)} colonnes")
print(f"\nAperçu des colonnes:")
print(df.columns.tolist())

In [ ]:
# Afficher les premières lignes
print("\n Aperçu des données:")
print(df.head())

print("\n Informations sur les données:")
print(df.info())

In [ ]:
# Catégoriser la colonne Position avant de la supprimer
if 'Position' in df.columns:
    print(f"\n Positions uniques trouvées:")
    print(df['Position'].value_counts())
    
    # Créer un mapping des positions vers des catégories numériques
    position_mapping = {
        'PG': 1,   # Point Guard
        'SG': 2,   # Shooting Guard
        'SF': 3,   # Small Forward
        'PF': 4,   # Power Forward
        'C': 5,    # Center
        'G': 6,    # Guard (générique)
        'F': 7,    # Forward (générique)
        'GF': 8,  # Guard-Forward
        '': 9     # Position inconnue
    }
    
    # Appliquer le mapping aux positions des joueurs
    df['position_category'] = df['Position'].map(position_mapping).fillna(0).astype(int)
    print(f"\n Colonne 'position_category' créée:")
    print("\n Colonne 'Position' non trouvée dans le dataset")

# Supprimer les colonnes inutiles pour la prédiction
columns_to_drop = ['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ABBREVIATION', 'Team', 'Salary', 'Season', 'Position', 'Rank']

# Vérifier quelles colonnes existent réellement
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
print(f"\n Colonnes à supprimer: {existing_columns_to_drop}")

df_clean = df.drop(columns=existing_columns_to_drop)

print(f"\n Dataset après nettoyage: {len(df_clean)} lignes, {len(df_clean.columns)} colonnes")
print(f"Colonnes restantes: {df_clean.columns.tolist()}")

In [ ]:
# Créer une colonne catégorielle pour les saisons à partir de 'Year'
# Extraire l'année de début (ex: '1999-00' -> 1999)
df_clean['season_start_year'] = df_clean['Year'].str.split('-').str[0].astype(int)

# Créer une colonne catégorielle (1 pour 1999-00, 2 pour 2000-01, etc.)
min_year = df_clean['season_start_year'].min()
df_clean['season_category'] = df_clean['season_start_year'] - min_year + 1

print(f"\n Catégories de saisons créées:")
print(f"   Année minimale: {min_year} (catégorie 1)")
print(f"   Année maximale: {df_clean['season_start_year'].max()} (catégorie {df_clean['season_category'].max()})")

In [ ]:
# Supprimer les colonnes temporaires et Year
df_clean = df_clean.drop(columns=['Year', 'season_start_year'])

# Gérer les valeurs manquantes
print(f"\n Valeurs manquantes par colonne:")
missing_values = df_clean.isnull().sum()
print(missing_values[missing_values > 0])

# Remplir les valeurs manquantes avec 0 (ou la médiane selon la colonne)
df_clean = df_clean.fillna(0)

print(f"\n Dataset final prêt pour l'entraînement: {len(df_clean)} lignes, {len(df_clean.columns)} colonnes")

In [ ]:
# Séparer les features (X) et la cible (y)
X = df_clean.drop(columns=['adjusted_salary'])
y = df_clean['adjusted_salary']

print(f" Variable cible: adjusted_salary")
print(f" Features: {len(X.columns)} colonnes")
print(f"   Nombre d'exemples: {len(X)}")
print(f"\n Statistiques de la variable cible:")
print(f"   Moyenne: ${y.mean():,.0f}")
print(f"   Médiane: ${y.median():,.0f}")
print(f"   Min: ${y.min():,.0f}")
print(f"   Max: ${y.max():,.0f}")

In [ ]:
# Diviser les données en ensembles d'entraînement et de test (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f" Division des données:")
print(f"   Entraînement: {len(X_train)} exemples ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Test: {len(X_test)} exemples ({len(X_test)/len(X)*100:.1f}%)")

In [ ]:
# Créer et entraîner le modèle Random Forest
print("Entraînement du modèle Random Forest...")
print("   (Cela peut prendre quelques minutes...)\n")

rf_model = RandomForestRegressor(
    n_estimators=100,      # Nombre d'arbres
    max_depth=20,          # Profondeur maximale des arbres
    min_samples_split=5,   # Nombre minimum d'échantillons pour diviser un nœud
    min_samples_leaf=2,    # Nombre minimum d'échantillons par feuille
    random_state=42,       # Pour la reproductibilité
    n_jobs=-1,             # Utiliser tous les cœurs CPU
    verbose=1              # Afficher la progression
)

# Entraîner le modèle
rf_model.fit(X_train, y_train)

print("\n Modèle entraîné avec succès!")

In [ ]:
# Faire des prédictions
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# Calculer les métriques de performance
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("="*60)
print(" RÉSULTATS DU MODÈLE RANDOM FOREST")
print("="*60)
print("\n Ensemble d'entraînement:")
print(f"   MAE (Mean Absolute Error):  ${train_mae:,.0f}")
print(f"   RMSE (Root Mean Squared Error): ${train_rmse:,.0f}")
print(f"   R² Score: {train_r2:.4f}")

print("\n Ensemble de test:")
print(f"   MAE (Mean Absolute Error):  ${test_mae:,.0f}")
print(f"   RMSE (Root Mean Squared Error): ${test_rmse:,.0f}")
print(f"   R² Score: {test_r2:.4f}")

print("\n Interprétation:")
print(f"   Le modèle se trompe en moyenne de ${test_mae:,.0f}")
print(f"   Il explique {test_r2*100:.1f}% de la variance des salaires")
print("="*60)

In [ ]:
# Importance des features
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n TOP 5 des variables les plus importantes:")
print(feature_importance.head(5).to_string(index=False))

# Visualiser les 5 features les plus importantes
plt.figure(figsize=(12, 8))
top_5 = feature_importance.head(5)
plt.barh(top_5['feature'], top_5['importance'])
plt.xlabel('Importance')
plt.title('Top 5 des variables les plus importantes pour prédire le salaire')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Visualiser les prédictions vs valeurs réelles
plt.figure(figsize=(12, 6))

# Graphique 1: Scatter plot
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Salaire réel ($)')
plt.ylabel('Salaire prédit ($)')
plt.title('Prédictions vs Valeurs réelles (Test)')
plt.ticklabel_format(style='plain', axis='both')

# Graphique 2: Distribution des erreurs
plt.subplot(1, 2, 2)
errors = y_test - y_test_pred
plt.hist(errors, bins=50, edgecolor='black')
plt.xlabel('Erreur de prédiction ($)')
plt.ylabel('Fréquence')
plt.title('Distribution des erreurs de prédiction')
plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
plt.ticklabel_format(style='plain', axis='x')

plt.tight_layout()
plt.show()

print(f"\n Statistiques des erreurs:")
print(f"   Erreur moyenne: ${errors.mean():,.0f}")
print(f"   Écart-type des erreurs: ${errors.std():,.0f}")